# Delivery Truck Packing Problem
There are about 100 typical box sizes used to ship consumer goods in North America.
Loading delivery trucks with items optimally is a complex combinatorial optimisation problem, especially when other constraints such as priority shipment status, order date, and truck weight/size limits are taken into account.


We discuss a simplified problem:
- 1 truck
- 1 package size
- upto 100 packages

We need to consider
- priority shipping status
- days since the order was placed
- delivery truck's weight capacity
- delivery truck's size capacity

\

---


This problem was discussed during [CQM release at Qubit 2021](https://youtu.be/jTDnGox0c9Y?t=3920)

## CQM Development Process
1. Write out the objective and constraints
2. Define the binary and/or integer variables
3. Convert the objective and constraints to math statements with binary and/or integer variables
4. Build the CQM model in Ocean from the individual objectives and constraints

### Objectives
1. Maximize the number of packages selected with priority shipping
2. Minimze the number of days the packages are in transit

### Constraints
1. Do not exceed the maximum number of packages that can fit on the truck (100)
2. Do not exceed the maximum weight capacity of the truck (3000 lbs, ~1360.7 kg)

## Mathematical Statements
\begin{equation}
  x_i = \left\{
    \begin{array}{ll}
        1 & \mbox{if package i is selected }  \\
        0 & \mbox{if package i is not selected}
    \end{array}
        \right.
\end{equation}
\
\begin{equation}
p_i = \mbox{priority of package, 1 for standard} \\
d_i = \mbox{number of days since package ordering} \\
\end{equation}

\begin{equation}
P   = \mbox{maximum packages that can fit on the truck} \\
W   = \mbox{maximum weight that the truck can carry}
\end{equation}
  
## Objectives
1. $$ min \ ( -   Σ _{i=0}^{N} p_i x_i) $$
2. $$ min \ (- Σ_{i=0}^{N} d_i x_i) $$

## Constraints
1. $$ \Sigma_{i=0}^{N} x_i = P $$
2. $$ \Sigma_{i=0}^{N} w_i x_i  \leq W $$

In [ ]:
# !pip install dwave.system

In [39]:
# imports
from dimod import ConstrainedQuadraticModel, Binary, quicksum
from dwave.system import LeapHybridCQMSampler
import numpy as np
import random
import itertools


In [40]:
num_items = 300
priority = [random.choice([1, 2, 3]) for _ in range(num_items)]
days_since_order = [random.choice(list(range(4))) for _ in range(num_items)]
weights = [random.randint(1, 100) for _ in range(num_items)] # between 200-700
max_weight = 3000
max_parcels = 100

In [41]:
# Initialize CQM
cqm = ConstrainedQuadraticModel()

# Create list of bin variables
bin_variables = [Binary(i) for i in range(num_items)]

# Objective 1: consider priority shipping
objective1 = -quicksum(priority[i] * bin_variables[i] for i in range(num_items))

# Objective 2: consider number of days since order was placed
objective2 = -quicksum(days_since_order[i] * bin_variables[i] for i in range(num_items))

# Add objectives to the CQM
cqm.set_objective(objective1 + objective2)


In [42]:
# Constraint 1: max parcels constraint
cqm.add_constraint(quicksum(bin_variables[i] for i in range(num_items)) == max_parcels,
                    label='max_parcels')

# Constraint 2: max capacity constraint
cqm.add_constraint(quicksum(weights[i] * bin_variables[i] for i in range(num_items)) <= max_weight,
                   label='max_capacity')


'max_capacity'

In [43]:
api_token = ''
cqm_sampler = LeapHybridCQMSampler(token=api_token)
sampleset = cqm_sampler.sample_cqm(cqm, time_limit=5, label='Truck Packing Problem')  # default: 5s

In [45]:
# first = next(itertools.filterfalse(lambda d: not getattr(d, 'is_feasible'), list(sampleset.data())))
# infeasible_sample = next(itertools.filterfalse(lambda d: getattr(d, 'is_feasible'), list(sampleset.data())))
# print(infeasible_sample.is_satisfied)

In [46]:
print(sampleset.info)

{'constraint_labels': ['max_parcels', 'max_capacity'], 'qpu_access_time': 32014, 'charge_time': 5000000, 'run_time': 5272072, 'problem_id': 'f50dce5e-e080-48dc-9d89-2c98a283b5d8', 'problem_label': 'Truck Packing Problem'}


In [47]:
# Process the results
feasible_sols = np.where(sampleset.record.is_feasible==True)
if len(feasible_sols[0]):
  first_feasible_sol = np.where(sampleset.record[feasible_sols[0][0]][0]==1)
  
  # Characterize the problem
  problem_array = np.zeros((3, 4)).astype(int)
  for i in range(num_items):
    problem_array[-1 * (priority[i]-3)][-1 * (days_since_order[i]-3)] += 1

  print('\n********************** PROBLEM **********************')
  print('                Days since order was placed            ')
  print('{:>5s}{:>5s}{:^5s}{:^5s}{:^5s}'.format('Priority |', '3', '2', '1', '0'))
  print('-' * 40)

  for i in range(3):
    print('{:>5s}{:>10s}{:^5s}{:^5s}{:^5s}'.format(str(-1 * (i-3)), str(problem_array[i][0]), 
                              str(problem_array[i][1]),str(problem_array[i][2]),str(problem_array[i][3])))
      
  solution_array = np.zeros((3, 4)).astype(int)
  for i in first_feasible_sol[0]:
    solution_array[-1 * (priority[i]-3)][-1 * (days_since_order[i]-3)] += 1

  print('\n********************** SOLUTION **********************')
  print('                Days since order was placed            ')
  print('{:>5s}{:>5s}{:^5s}{:^5s}{:^5s}'.format('Priority |', '3', '2', '1', '0'))
  print('-' * 40)

  for i in range(3):
    print('{:>5s}{:>10s}{:^5s}{:^5s}{:^5s}'.format(str(-1 * (i-3)), str(solution_array[i][0]), 
                              str(solution_array[i][1]),str(solution_array[i][2]),str(solution_array[i][3])))
      

  print('Total number of selected items:', len(first_feasible_sol[0]))
  print('Total weight of selected items:', sum([weights[i] for i in first_feasible_sol[0]]))


********************** PROBLEM **********************
                Days since order was placed            
Priority |    3  2    1    0  
----------------------------------------
    3        26 25   37   17  
    2        17 35   29   27  
    1        26 21   23   17  

********************** SOLUTION **********************
                Days since order was placed            
Priority |    3  2    1    0  
----------------------------------------
    3        23 18   11    4  
    2        12 12    1    0  
    1        14  5    0    0  
Total number of selected items: 100
Total weight of selected items: 2994
